### extra

In [4]:
#np.set_printoptions(threshold=np.inf) #check full array lenght

#pd.read_csv("attachment(1)_alltictactoe.csv", header = None) #check csv structure or preview

#TTT2 = np.array([[1, 3, 5],[2, 3, np.nan],[np.nan, np.nan, np.nan]]) #possible TTT format

## all 9! games

In [5]:
import numpy as np

def ALL_tic_tac_toe():
    """
    generate all tic-tac-toe games but,
        doesn't care who starts (X or O)
        doesn't care if theres a win, just fills all the nine squares
        impar numbers stand for the starter moves (X or O) and par numbers stand for the other player
        it's justs all the 9! games
    """
    move = 0
    ith_move = np.array([[0,0,0 , 0,0,0 , 0,0,0], [0,0,0 , 0,0,0 , 0,0,0]])
    
    move = 1
    while move < 9:
        all_games = np.array([0,0,0 , 0,0,0 , 0,0,0])
        list_all_games = [all_games]
        for j in range(1, len(ith_move)):
            for i in range(9):
                if ith_move[j][i] != 0:
                    pass
                else:
                    ith_move[j][i] = move
                    list_all_games.append(ith_move[j].copy())
                    ith_move[j][i] = 0
        ith_move = np.vstack(list_all_games)
        #print("Move " + str(move) + " - OK")
        move += 1    

    move = 9
    last_move = np.where(ith_move[1:] == 0, 9, ith_move[1:])
    #print("Move " + str(move) + " - OK")

    np.savetxt("attachment(1)_alltictactoe.csv", last_move, delimiter=",", fmt="%d")

    return "attachment(1) saved", len(last_move)

ALL_tic_tac_toe()

('attachment(1) saved', 362880)

## true games - checking for wins

In [6]:
import numpy as np

def TRUE_tic_tac_toe():
    """
    remove all the repated games which have wins and bla bla bla
    """
    def is_there_a_win(ttt):
        # convert 1D array into 2D array (3 by 3)
        TTT = ttt.reshape(3,3)

        # lines to verify for wins (lines, columns, diagonals)
        lines = np.vstack([TTT, np.transpose(TTT), np.diag(TTT), np.diag(np.fliplr(TTT))])
        
        # check sums to see if there is a winner
        X_vs_O = np.where(lines % 2 == 1, 1, -1)
        sums = np.sum(X_vs_O, axis=1)

        # someone won
        if 3 in np.abs(sums):
            # find the winning move
            last_move = np.max(lines, axis=1)
            last_move[np.abs(sums) != 3] = 10
            return True, min(last_move)

        # draw game
        else:
            return False, 0
        

    with open("attachment(1)_alltictactoe.csv", "r") as csvfile:
        rows = [line.strip().split(',') for line in csvfile]

    new_games = []
    for row in rows:
        ttt = np.array(row).astype(float)
        win = is_there_a_win(ttt)
        if win[0]:
            ttt[ttt > win[1]] = 0
            new_games.append(ttt)
        else:
            new_games.append(ttt)

    processed_games = np.vstack(new_games)
    unique_games = np.unique(processed_games, axis=0)

    np.savetxt("attachment(2)_truetictactoe.csv", unique_games, delimiter=",", fmt="%d")

    return "attachment(2) saved", len(unique_games)

TRUE_tic_tac_toe()

('attachment(2) saved', 255168)

# symmertryes

In [7]:
import numpy as np

def SYMMETRIES_tic_tac_toe(n = 3, savefile = True):
    """
    simetrias bla bla
    n = 3 -> 3 by 3 square
    """

    ############################################################################
    def _symmetry_squares(n = n):
        """ ex.: if there's vertical symmetry which squares are valid moves """
        middle_line = n//2 + n%2

        vertical = []
        for row in range(n):
            for col in range(middle_line):
                vertical.append(row * n + col)

        horizontal = []
        for row in range(middle_line):
            for col in range(n):
                horizontal.append(row * n + col)
        
        diagonal = []
        anti_diagonal = [] 
        for row in range(n):
            for col in range(n):
                if row <= col:
                    diagonal.append(row * n + col)
                
                if col <= (n-1) - row:
                    anti_diagonal.append(row * n + col)

        return vertical, horizontal, diagonal, anti_diagonal
    ############################################################################


    ############################################################################
    symmetry_squares = _symmetry_squares(n)
    
    def _valid_moves(TTT, n = n, symmetry_squares = symmetry_squares):
        middle_line = n//2 + n%2

        # define all 4 axes of symmetry and verify them (True or False)
        vertical = np.all([np.transpose(TTT)[i] == np.transpose(TTT)[-i-1] for i in range(middle_line)])
        horizontal = np.all([TTT[i] == TTT[-i-1] for i in range(middle_line)])
        diagonal = np.all(TTT == np.transpose(TTT))
        anti_diagonal = np.all(np.fliplr(TTT) == np.transpose(np.fliplr(TTT)))
        axes_of_symmetry = vertical, horizontal, diagonal, anti_diagonal

        # which are the valid moves
        all_possible_moves = set(range(n*n))
        invalid_moves = []
        for i in range(4):
            if axes_of_symmetry[i]:
                invalid_moves_i = all_possible_moves - set(symmetry_squares[i])
                invalid_moves += invalid_moves_i
        valid_moves = all_possible_moves - set(invalid_moves)

        return list(valid_moves)
    ############################################################################


    ############################################################################
    def _its_a_win(TTT, n = n):
        lines = np.vstack([TTT, np.transpose(TTT), np.diag(TTT), np.diag(np.fliplr(TTT))])
        # if 0 -> 0 ; if impar -> 1 ; if par -> -1   
        X_vs_O = np.where(lines == 0, 0, np.where(lines % 2 == 1, 1, -1))
        sums = np.sum(X_vs_O, axis=1)
        return n in np.abs(sums)
    ############################################################################
    
    TTT_moves = np.zeros(n**2)
    TTT_symmetries = []

    ongoing_games1 = [TTT_moves]
    move = 1
    while move < n**2:
        ongoing_games2 = []
        for game in ongoing_games1:
            # verify possible moves
            valid_moves = _valid_moves(game.reshape(n,n))
            # play them
            for valid_move in valid_moves:
                if game[valid_move] != 0:
                    pass
                else:
                    game[valid_move] = move
                    # verify if its a winning move
                    if _its_a_win(game.reshape(n,n)):
                        TTT_symmetries.append(game.copy())
                    else:
                        ongoing_games2.append(game.copy())
                        
                    game[valid_move] = 0
        # next move
        move += 1
        ongoing_games1 = ongoing_games2
        
    # last move
    move = n**2
    for game in ongoing_games1:
        # was move (n**2 - 1) a winning move ?
        if _its_a_win(game.reshape(n,n)):
            TTT_symmetries.append(game.copy())
        else:
            TTT_symmetries.append(np.where(game == 0, n**2, game).copy())

    if savefile:
        np.savetxt("attachment(3)_symmetriestictactoe.csv", TTT_symmetries, delimiter=",", fmt="%d")
        return "attachment(3) saved", len(TTT_symmetries)
    else:
        return TTT_symmetries, len(TTT_symmetries)


SYMMETRIES_tic_tac_toe()

('attachment(3) saved', 31896)